In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------


Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 9

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v07"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v07 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V09/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V09/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M08D23 UTC 16:52


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/engine/default.py", line 457, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL error: decryption failed or bad record mac



/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111011_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//223940_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//155562_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//172497_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132895_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//181976_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//122784_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//153021_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//157807_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//162203_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//145102_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115801_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//142625_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//127443_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//155571_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//122810_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//153027_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//158115_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115808_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//127450_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//145109_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//181994_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132941_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//232160_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111019_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//142634_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//131909_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//172525_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//184000_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//232240_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//145207_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//158142_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//122845_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//155579_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//127467_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132948_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132170_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//172542_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//162444_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//142643_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//116006_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111046_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//232302_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//225117_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//145404_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//127484_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132224_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//158150_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111081_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//172549_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//132956_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//155617_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//162461_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117102_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//153043_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//122854_V09.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//822478_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//662903_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//726014_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742987_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//642456_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914805_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//812202_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911101_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//753304_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//831955_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772350_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//723010_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742426_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//652206_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772403_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//831980_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742995_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911211_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//652303_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//664010_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//723072_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742443_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//712900_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//753402_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//822496_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//782706_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911212_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//726040_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742450_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//822514_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772601_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//642473_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//743005_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//723079_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//782804_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//812308_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//664028_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911240_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//652402_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//726075_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//713225_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//832108_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//713250_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//726083_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//723115_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//753508_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742486_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//822541_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//751203_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772609_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//642482_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911269_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//832424_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//652500_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//812471_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//713260_V09.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//753626_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//731110_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772808_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//751402_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//832444_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//812480_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911401_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//782993_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//822550_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//742496_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//731120_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//772809_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//753627_V09.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//751403_V09.csv
/volumes/data/Y2018M

In [14]:
#!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111011_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111012_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111013_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111014_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111016_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111015_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111017_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111018_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//111019_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112114_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112120_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112150_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112140_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112130_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112160_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112180_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112200_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//112190_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114447_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114448_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114449_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114450_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114460_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114601_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114470_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114490_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//114480_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115427_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115428_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115429_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115430_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115440_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115451_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115452_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115453_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//115454_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117106_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117107_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117108_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117109_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117201_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117202_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117203_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117204_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//117205_V09.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863017_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863018_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863019_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863020_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863030_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863040_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863050_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863060_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//863070_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911540_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911550_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911560_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911570_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911580_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911591_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911592_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911593_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//911594_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912556_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912557_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912558_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912559_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912560_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912571_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912572_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912573_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//912574_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913530_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913540_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913550_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913561_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913562_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913563_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913564_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913565_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//913566_V09.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914802_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914803_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914804_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914806_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914805_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914807_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914808_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914809_V09.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V09//914900_V09.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:23:35.132388


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388

